In [1]:
import os
import glob

# Path to your dataset folder
# Each class should be in a separate folder inside this directory
dataset_path = "dataset"

# List all subfolders (classes)
classes = [d for d in os.listdir(dataset_path) if os.path.isdir(os.path.join(dataset_path, d))]

print("Found classes:", classes)
print("Total classes:", len(classes))
# Count images in each class
for cls in classes:
    class_path = os.path.join(dataset_path, cls)
    # Assuming common image extensions
    image_files = glob.glob(os.path.join(class_path, "*.[pjPJ]*[ngNG]*"))  # jpg, jpeg, png
    print(f"Class '{cls}' has {len(image_files)} images")


Found classes: ['Acne And Rosacea Photos', 'Malignant', 'Fu Athlete Foot', 'Eczema Photos', 'Fu Ringworm', 'Benign', 'Exanthems And Drug Eruptions', 'Lupus And Other Connective Tissue Diseases', 'Ba Impetigo', 'Bullous Disease Photos', 'Nail Fungus And Other Nail Disease', 'Light Diseases And Disorders Of Pigmentation', 'Atopic Dermatitis Photos', 'Actinic Keratosis Basal Cell Carcinoma And Other Malignant Lesions', 'Melanoma Skin Cancer Nevi And Moles', 'Cellulitis Impetigo And Other Bacterial Infections', 'Hair Loss Photos Alopecia And Other Hair Diseases', 'Herpes Hpv And Other Stds Photos', 'Fu Nail Fungus', 'Ba  Cellulitis']
Total classes: 20
Class 'Acne And Rosacea Photos' has 6837 images
Class 'Malignant' has 6762 images
Class 'Fu Athlete Foot' has 8054 images
Class 'Eczema Photos' has 6715 images
Class 'Fu Ringworm' has 8129 images
Class 'Benign' has 6459 images
Class 'Exanthems And Drug Eruptions' has 7750 images
Class 'Lupus And Other Connective Tissue Diseases' has 7734 imag

In [2]:
from tensorflow.keras import mixed_precision
mixed_precision.set_global_policy("mixed_float16")

In [3]:
import os

# Suppress TensorFlow Python logs
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "3"

# Suppress ML Compute / Metal debug info
os.environ["TF_METAL_ENABLE_LOGGING"] = "0"
os.environ["TF_GPU_THREAD_MODE"] = "gpu_private"

import tensorflow as tf
tf.debugging.set_log_device_placement(False)


In [4]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import os

In [5]:
DATA_ROOT = "dataset"   # <-- change to your dataset folder
IMG_SIZE = 224
BATCH_SIZE = 32
EPOCHS = 15
VAL_SPLIT = 0.15
SEED = 1337

In [6]:
train_ds = tf.keras.utils.image_dataset_from_directory(
    DATA_ROOT,
    validation_split=VAL_SPLIT,
    subset="training",
    seed=SEED,
    image_size=(IMG_SIZE, IMG_SIZE),
    batch_size=BATCH_SIZE,
)

val_ds = tf.keras.utils.image_dataset_from_directory(
    DATA_ROOT,
    validation_split=VAL_SPLIT,
    subset="validation",
    seed=SEED,
    image_size=(IMG_SIZE, IMG_SIZE),
    batch_size=BATCH_SIZE,
)

class_names = train_ds.class_names
num_classes = len(class_names)
print(f"Classes: {num_classes}")

Found 150686 files belonging to 20 classes.
Using 128084 files for training.
Found 150686 files belonging to 20 classes.
Using 22602 files for validation.
Classes: 20


In [7]:
AUTOTUNE = tf.data.AUTOTUNE
train_ds = train_ds.shuffle(1000).prefetch(buffer_size=AUTOTUNE)
val_ds   = val_ds.prefetch(buffer_size=AUTOTUNE)

In [8]:
data_augmentation = keras.Sequential([
    layers.RandomFlip("horizontal"),
    layers.RandomRotation(0.05),
    layers.RandomZoom(0.1),
])

In [9]:
base_model = keras.applications.EfficientNetB0(
    include_top=False,
    weights="weights_mobilenet_v3_large_224_1.0_float_no_top_v2.h5",
    input_shape=(IMG_SIZE, IMG_SIZE, 3),
    pooling="avg",
)

In [10]:
base_model.trainable = False  # freeze for transfer learning

inputs = keras.Input(shape=(IMG_SIZE, IMG_SIZE, 3))
x = data_augmentation(inputs)
x = keras.applications.efficientnet.preprocess_input(x)
x = base_model(x, training=False)
x = layers.Dropout(0.3)(x)
outputs = layers.Dense(num_classes, activation="softmax")(x)


In [11]:
model = keras.Model(inputs, outputs)

In [12]:
model.compile(
    optimizer=keras.optimizers.Adam(learning_rate=3e-4),
    loss=keras.losses.SparseCategoricalCrossentropy(),
    metrics=["accuracy"],
)


In [13]:
callbacks = [
    keras.callbacks.EarlyStopping(patience=3, restore_best_weights=True),
    keras.callbacks.ReduceLROnPlateau(factor=0.5, patience=2),
]

In [14]:
history = model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=EPOCHS,
    callbacks=callbacks,
)

Epoch 1/15
4003/4003 ━━━━━━━━━━━━━━━━━━━━ 2459s 608ms/step - accuracy: 0.5545 - loss: 1.4807 - val_accuracy: 0.6543 - val_loss: 1.1290 - learning_rate: 3.0000e-04
Epoch 2/15
4003/4003 ━━━━━━━━━━━━━━━━━━━━ 2413s 597ms/step - accuracy: 0.6361 - loss: 1.1659 - val_accuracy: 0.6763 - val_loss: 1.0278 - learning_rate: 3.0000e-04
Epoch 3/15
4003/4003 ━━━━━━━━━━━━━━━━━━━━ 2408s 594ms/step - accuracy: 0.6526 - loss: 1.0976 - val_accuracy: 0.6974 - val_loss: 0.9741 - learning_rate: 3.0000e-04
Epoch 4/15
4003/4003 ━━━━━━━━━━━━━━━━━━━━ 2396s 592ms/step - accuracy: 0.6620 - loss: 1.0606 - val_accuracy: 0.7052 - val_loss: 0.9383 - learning_rate: 3.0000e-04
Epoch 5/15
4003/4003 ━━━━━━━━━━━━━━━━━━━━ 2385s 589ms/step - accuracy: 0.6650 - loss: 1.0432 - val_accuracy: 0.7103 - val_loss: 0.9169 - learning_rate: 3.0000e-04
Epoch 6/15
4003/4003 ━━━━━━━━━━━━━━━━━━━━ 2374s 586ms/step - accuracy: 0.6697 - loss: 1.0279 - val_accuracy: 0.7137 - val_loss: 0.9021 - learning_rate: 3.0000e-04
Epoch 7/15
4003/4003 ━

In [15]:
base_model.trainable = True
model.compile(
    optimizer=keras.optimizers.Adam(1e-5),
    loss=keras.losses.SparseCategoricalCrossentropy(),
    metrics=["accuracy"],
)

fine_history = model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=5,
    callbacks=callbacks,
)

Epoch 1/5
4003/4003 ━━━━━━━━━━━━━━━━━━━━ 15234s 4s/step - accuracy: 0.5478 - loss: 1.4910 - val_accuracy: 0.6746 - val_loss: 1.0025 - learning_rate: 1.0000e-05
Epoch 2/5
4003/4003 ━━━━━━━━━━━━━━━━━━━━ 13007s 3s/step - accuracy: 0.6793 - loss: 0.9879 - val_accuracy: 0.7359 - val_loss: 0.8214 - learning_rate: 1.0000e-05
Epoch 3/5
4003/4003 ━━━━━━━━━━━━━━━━━━━━ 15136s 4s/step - accuracy: 0.7356 - loss: 0.8125 - val_accuracy: 0.7744 - val_loss: 0.7046 - learning_rate: 1.0000e-05
Epoch 4/5
4003/4003 ━━━━━━━━━━━━━━━━━━━━ 18158s 5s/step - accuracy: 0.7761 - loss: 0.6863 - val_accuracy: 0.7987 - val_loss: 0.6200 - learning_rate: 1.0000e-05
Epoch 5/5
4003/4003 ━━━━━━━━━━━━━━━━━━━━ 17254s 4s/step - accuracy: 0.8073 - loss: 0.5867 - val_accuracy: 0.8284 - val_loss: 0.5334 - learning_rate: 1.0000e-05


In [16]:
model.save("skin_disease_model_tf.h5")
model.save("skin_disease_model_tf.keras")